In [8]:
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
import boto3
import sys
import os

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [27]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.schema.document import Document

text_file = open('Biographical.txt', 'r', encoding='utf-8', errors='ignore')
strs = text_file.readlines()
text_file.close()

docs = []

for str in strs:
    docs.append(Document(page_content=str))  

vectorstore_faiss_jonabot = FAISS.from_documents(
    documents=docs,
    embedding = br_embeddings, 
)

print(f"vectorstore_faiss_jonabot:created={vectorstore_faiss_jonabot}::")

vectorstore_faiss_jonabot:created=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E356DF6A60>::


In [20]:
from langchain.llms.bedrock import Bedrock
modelId = "amazon.titan-tg1-large"
titan_llm = Bedrock(model_id=modelId, client=boto3_bedrock)
titan_llm.model_kwargs = {'temperature': 0.1, "maxTokenCount": 4096}

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_jonabot)
print_ww(wrapper_store_faiss.query("Favorite Bar", llm=titan_llm))



Crown Alley

